In [ ]:
from github import Github
import getpass
import time
import pandas as pd
import csv
import langdetect
import base64
import os

g = Github(getpass.getpass("Enter access token:"), per_page=100)

Retrieves repository readme files and saves them in a folder.

In [ ]:
languages = ["Java", "Python"]

readme_folder = "readmes"

if not os.path.exists(readme_folder):
    os.makedirs(readme_folder)

for language in languages:

    candidate_repos = pd.read_csv(f"candidate_repos_{language.lower()}.csv", delimiter=";")
    print(f"Retrieving the readme files for {candidate_repos.shape[0]} repositories in {language}...")
    
    collected = 0
        
    for row_id, row in candidate_repos.iterrows():
        readme_file = f"{readme_folder}/{row.OWNER}_{row.NAME}_readme.md"
        if not os.path.exists(readme_file):
            repo = g.get_repo(f"{row.OWNER}/{row.NAME}")
            time.sleep(0.5)
            try:
                readme = base64.b64decode(repo.get_readme()._rawData["content"].encode()).decode("UTF-8")
                with open(readme_file, "w", encoding="utf-8") as file:
                    file.write(readme)
                collected += 1
            except Exception as e:
                print(f"{row.OWNER}/{row.NAME}: {e}")
                continue
                
    print(f"{collected} new readme files were collected.")

In [ ]:
# !jupyter nbconvert --to script 2_repo_readme_collector.ipynb # converts to .py file